Importing necessary modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
import sys

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir Data
!cp "/content/drive/MyDrive/DL_LAB/ASG1/diabetes.csv" "./Data"
!cp "/content/drive/MyDrive/DL_LAB/ASG1/User_Data.csv" "./Data"
!cp "/content/drive/MyDrive/DL_LAB/ASG1/50_Startups.csv" "./Data"

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def predict(X, w, reg="linear"):
    y_pred = X @ w
    if(reg == 'logistic'):
      y_pred = 1/(1 + np.exp(-y_pred)) # apply sigmoid fun
    return y_pred
    
def fit(X, y, lr,max_epochs ,reg):
    
    X = np.c_[np.ones((X.shape[0], 1)), X] # adding bias column
    precision = 0.00000000001 #until difference in w_k & w_k+1 is closed to zero
    change_in_w = 1
    w = np.random.rand(X.shape[1],1) #Randomly initializing the weights
    iter = 0

    while( iter <= max_epochs) and change_in_w > precision:
        prev_w = w
        
        if(reg == 'linear'):
            y_pred = predict(X,w, 'linear')
        elif(reg == 'logistic'):
            y_pred = predict(X,w, 'logistic')

        dE = X.T @ (y_pred - y) 
        
        w = w - (lr*dE)
        iter += 1
        change_in_w = abs(np.sum(w) - np.sum(prev_w))
    return w
 

### **Use User database and predict that a user will purchase the company’s newly launched product or not.**

In [ ]:
users_df=pd.read_csv('Data/User_Data.csv')
print("Shape :",users_df.shape)
users_df.head()

Shape : (400, 5)


,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [ ]:
users_df.dropna() # Drop the rows which contains null values

features = users_df.iloc[:, :-1] # all col except last are features
target = users_df.iloc[:, -1].values.reshape((features.shape[0], 1)) # last col

#replace col state with categorical cols
features.head()

,User ID,Gender,Age,EstimatedSalary
0,15624510,Male,19,19000
1,15810944,Male,35,20000
2,15668575,Female,26,43000
3,15603246,Female,27,57000
4,15804002,Male,19,76000


In [ ]:
target[:8]

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1]])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features[['Age', 'EstimatedSalary']],target,test_size=0.2,random_state=27)

In [ ]:
# Normalizing the data

norm = MinMaxScaler().fit(X_train)

X_train_normalized = norm.transform(X_train)

X_test_normalized = norm.transform(X_test)

**From sklearnlogistic regression**

In [ ]:
log_reg_obj = LogisticRegression()
log_reg_obj.fit(X_train_normalized,y_train.ravel()) #ravel will convert that array shape to (n, ) (i.e. flatten it)

LogisticRegression()

In [ ]:
print("Training set accuracy:",log_reg_obj.score(X_train_normalized,y_train)*100)
print("Testing set accuracy:",log_reg_obj.score(X_test_normalized,y_test)*100)

Training set accuracy: 82.8125
Testing set accuracy: 80.0


**From scratch**

In [ ]:
X_train_normalized.shape, y_train.shape

((320, 2), (320, 1))

In [ ]:
w = fit(X_train_normalized, y_train, 0.001, 10000 ,"logistic")

X_test_norm_withBias = np.c_[np.ones((X_test_normalized.shape[0], 1)), X_test_normalized]
y_pred = predict(X_test_norm_withBias, w, "logistic")

In [ ]:
y_pred[:5]

array([[0.00766396],
       [0.56922949],
       [0.23841517],
       [0.01205873],
       [0.17964436]])

In [ ]:
y_pred_classified = [1 if y_pred[i]>0.5 else 0 for i in range(y_pred.shape[0]) ]

In [ ]:
err = y_pred_classified- (y_test).flatten()

In [ ]:
count=0
for i in err:
  if i ==0: count+=1 

acc = 1/len(err) * count
print("accuracy in scratch:",acc*100,"%")

accuracy in scratch: 85.00000000000001 %


###**Use Pima diabetes dataset and classify them as diabetes or no diabetes based on features available.**

In [ ]:
diabetes_df=pd.read_csv('Data/diabetes.csv')
print("Shape :",diabetes_df.shape)
diabetes_df.head()

Shape : (768, 9)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
features2 = diabetes_df.iloc[:, :-1] # all col except last are features
target2 = diabetes_df.iloc[:, -1].values.reshape((features2.shape[0], 1)) # last col

X_train2, X_test2, y_train2, y_test2 = train_test_split(features2,target2,test_size=0.2,random_state=27)

norm = MinMaxScaler().fit(X_train2)

X_train_normalized2 = norm.transform(X_train2)

X_test_normalized2 = norm.transform(X_test2)

**From sklearn logistic regression**

In [ ]:
log_reg_obj = LogisticRegression()
log_reg_obj.fit(X_train_normalized2,y_train2.ravel()) #.ravel will convert that array shape to (n, ) (i.e. flatten it)
print("Training set accuracy:",log_reg_obj.score(X_train_normalized2,y_train2)*100,"%")
print("Testing set accuracy:",log_reg_obj.score(X_test_normalized2,y_test2)*100,"%")


Training set accuracy: 77.36156351791531 %
Testing set accuracy: 79.22077922077922 %


**From scratch**

In [ ]:
w = fit(X_train_normalized2, y_train2, 0.001, 10000 ,"logistic")

X_test_norm_withBias2 = np.c_[np.ones((X_test_normalized2.shape[0], 1)), X_test_normalized2]
y_pred2 = predict(X_test_norm_withBias2, w, "logistic")
y_pred_classified2 = [1 if y_pred2[i]>0.5 else 0 for i in range(y_pred2.shape[0]) ]

err = y_pred_classified2- (y_test2).flatten()
count=0
for i in err:
  if i ==0: count+=1 

acc = 1/len(err) * count
print("accuracy in scratch:",acc*100,"%")

accuracy in scratch: 74.67532467532469 %


##**Predict the profit values on 50_startups dataset for each company.**

In [ ]:
startups_df=pd.read_csv('Data/50_Startups.csv')
print("Shape :",startups_df.shape)
startups_df.head()

Shape : (50, 5)


,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [ ]:
features3= startups_df.iloc[:, :-1] # all col except last are features
target3 = startups_df.iloc[:, -1].values.reshape((features3.shape[0], 1)) # last col

# convert state col into categorical col
state = pd.get_dummies(features3, columns=['State']) # to change prefix #, drop_first=True

#replace col state with categorical cols
features3 = features3.drop('State', axis=1)
features3 = pd.concat([state], axis=1)

X_train3, X_test3, y_train3, y_test3 = train_test_split(features3[['R&D Spend', 'Administration', 'Marketing Spend']],target3,test_size=0.2,random_state=27)

# fit scaler on training data
norm = MinMaxScaler().fit(X_train3)

# transform training data
X_train_normalized3 = norm.transform(X_train3)

# transform testing dataabs
X_test_normalized3 = norm.transform(X_test3)

**From sklearn Linear regression**

In [ ]:
lin_reg_obj = LinearRegression()
lin_reg_obj.fit(X_train_normalized3,y_train3.ravel()) #.ravel will convert that array shape to (n, ) (i.e. flatten it)
print("Training set accuracy:",lin_reg_obj.score(X_train_normalized3,y_train3)*100,'%')
print("Testing set accuracy:",lin_reg_obj.score(X_test_normalized3,y_test3)*100,'%')

Training set accuracy: 95.23004904379036 %
Testing set accuracy: 92.86264675370695 %


**From scratch**

In [ ]:
w = fit(X_train_normalized3, y_train3, 0.001, 20000 ,"linear")

X_test_norm_withBias3 = np.c_[np.ones((X_test_normalized3.shape[0], 1)), X_test_normalized3]
y_pred3 = predict(X_test_norm_withBias3, w, "linear")

In [ ]:
r2Score = r2_score(y_test3, y_pred3)
print("R2 score of model is :" ,r2Score)

R2 score of model is : 0.9286036638574566
